In [58]:
## import of the pandas library
## questions for myself: could this be done recursively? Get the books on one level and then go to other levels? Transitive...
import pandas as pd
import pickle

In [59]:
with open('data/interim/books_data.pkl', 'rb') as f:
    books_data = pickle.load(f)

with open('data/interim/ratings_data.pkl', 'rb') as f:
    ratings_data= pickle.load(f)

with open('data/interim/users_data.pkl', 'rb') as f:
    users_data= pickle.load(f)

with open('data/interim/books_ratings.pkl', 'rb') as f:
    books_ratings= pickle.load(f)

books_data


,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
0,0195153448,MARK P. O. MORFORD,CLASSICAL MYTHOLOGY,2002,OXFORD UNIVERSITY PRESS,EN
1,0002005018,RICHARD BRUCE WRIGHT,CLARA CALLAN,2001,HARPERFLAMINGO CANADA,EN
2,0060973129,CARLO D'ESTE,DECISION IN NORMANDY,1991,HARPERPERENNIAL,EN
3,0374157065,GINA BARI KOLATA,FLU THE STORY OF THE GREAT INFLUENZA PANDEMIC ...,1999,FARRAR STRAUS GIROUX,EN
4,0393045218,E. J. W. BARBER,THE MUMMIES OF URUMCHI,1999,W. W. NORTON &AMP; COMPANY,EN
...,...,...,...,...,...,...
271355,0440400988,PAULA DANZIGER,THERE'S A BAT IN BUNK FIVE,1988,RANDOM HOUSE CHILDRENS PUB (MM),EN
271356,0525447644,TERI SLOAT,FROM ONE TO ONE HUNDRED,1991,DUTTON BOOKS,EN
271357,006008667X,CHRISTINE WICKER,LILY DALE THE TRUE STORY OF THE TOWN THAT TAL...,2004,HARPERSANFRANCISCO,EN
271358,0192126040,PLATO,REPUBLIC WORLD'S CLASSICS,1996,OXFORD UNIVERSITY PRESS,EN


In [60]:
def recommendation (BookIsLiked, BookTitle, BookAuthor):

    ## find the lord of the Rings books
    filtered_books = books_data.loc[books_data["Book-Title"].str.contains(BookTitle) & books_data["Book-Author"].str.contains(BookAuthor)]
    filtered_books

    ## getting the users that like Lord of the Rings
    users_filtered = ratings_data.loc[(ratings_data['ISBN'].isin(filtered_books['ISBN'])) & (ratings_data['Book-Rating'] > BookIsLiked)]
    lord_readers = pd.unique(users_filtered['User-ID'])
    lord_readers

    ## getting the ISBN and Book-Ratings of the books that Lord of the Rings lovers also read
    books = ratings_data[ratings_data['User-ID'].isin(lord_readers)].reindex(columns = ['ISBN', 'Book-Rating'])
    ##books

    ## keeping only the books where there is more than 5 ratings
    count = books.groupby('ISBN')['Book-Rating'].count().reset_index(name = 'Count')
    count = count.loc[count["Count"] > 5]

    ## keeping only the books where the mean of the ratings is bigger than BookIsLiked, but the book should not be Lord of the Rings book
    ratings = pd.merge(left=ratings_data,right=count, left_on="ISBN", right_on="ISBN")
    ratings = ratings.groupby('ISBN')['Book-Rating'].mean().reset_index(name = 'MeanRating').sort_values(by='MeanRating', ascending=False)
    ratings = ratings.loc[ratings["MeanRating"] > BookIsLiked]
    ratings = ratings[ratings.ISBN.isin(filtered_books["ISBN"])==False]

    ##ratings
    return books_data[books_data['ISBN'].isin(ratings['ISBN'])]

In [64]:
## defining variables
BookIsLiked = 6 ## what does it mean that I like the book? How high the rating must be?
BookTitle = 'SHINING'
BookAuthor = 'KING'

In [65]:
recommendation (BookIsLiked, BookTitle, BookAuthor)

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
3357,0385199570,STEPHEN KING,THE STAND THE COMPLETE AND UNCUT EDITION,1990,DOUBLEDAY BOOKS,EN
